In [1]:
import torch
torch.manual_seed(42)
from torch import nn
from torch.utils.data import DataLoader,Dataset

In [2]:
from privacyflow.configs import path_configs
from privacyflow.datasets import sleep_dataset
from privacyflow.preprocessing import sleep_preprocess
from privacyflow.models import sleep_model_base

In [3]:
#Check if GPU is available
if torch.cuda.is_available():
    print("GPU will be used")
    device = torch.device('cuda')
else:
    print("No GPU available")
    device = torch.device('cpu')

GPU will be used


## Data Prep

Der Sleep-Datensatz ist eine einzelne .csv Datei. Diese wird mittels einer Preprocessing-Methode bereinigt und in Train-Val-Test gesplittet.
Diese können anschließend als PyTorch Dataset genutzt werden.

In [4]:
sleep_preprocess.preprocess_sleep_data()

In [5]:
train_dataset = sleep_dataset.SleepDataset(mode="train")
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True
)

val_dataset = sleep_dataset.SleepDataset(mode="val")
val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=32,
    shuffle=False
)

test_dataset = sleep_dataset.SleepDataset(mode="test")
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=32,
    shuffle=False
)

## Model

In [6]:
model = sleep_model_base.SleepModelBase(12,3)
model = model.to(device)

## Training

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)

In [11]:
for epoch in range(12):
    model.train()
    epoch_loss = 0.0
    for batch in train_dataloader:
        inputs,labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    model.eval()
    val_loss = 0.0
    for batch in val_dataloader:
        inputs,labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        val_loss += loss.item()

    print(f"Epoch: {epoch+1}, Train Loss:{epoch_loss/len(train_dataloader)}, Val Loss:{val_loss/len(val_dataloader)}")


Epoch: 1, Train Loss:0.6849068254232407, Val Loss:0.8670949637889862
Epoch: 2, Train Loss:0.632809916138649, Val Loss:0.8294864892959595
Epoch: 3, Train Loss:0.5712380111217499, Val Loss:0.6637380421161652
Epoch: 4, Train Loss:0.5942749053239822, Val Loss:0.8508903384208679
Epoch: 5, Train Loss:0.6161019146442414, Val Loss:0.5757983922958374
Epoch: 6, Train Loss:0.7206846237182617, Val Loss:0.9786092042922974
Epoch: 7, Train Loss:0.697091567516327, Val Loss:0.8197578489780426
Epoch: 8, Train Loss:0.6325776875019073, Val Loss:0.6183421611785889
Epoch: 9, Train Loss:0.6211926579475403, Val Loss:0.8949172496795654
Epoch: 10, Train Loss:0.6117232412099838, Val Loss:0.6472404301166534
Epoch: 11, Train Loss:0.6231926083564758, Val Loss:1.1291579008102417
Epoch: 12, Train Loss:0.7767252922058105, Val Loss:1.1961228847503662


In [12]:
num_correct = 0.0
model.eval()
for batch in test_dataloader:
    inputs,labels = batch
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, predicted = torch.max(outputs,1)
    num_correct += (predicted == labels).sum()

accuracy = num_correct/len(test_dataset)
print(accuracy)

tensor(0.7105, device='cuda:0')
